#### 酒店集群推荐比赛代码

In [25]:
import pandas as pd
import seaborn as sns
import numpy as np
sns.set_style('whitegrid')

destinations = pd.read_csv("destinations.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

查看训练数据的前几行数据以及训练数据的维度

In [26]:
print(train.head())
print(train.shape)

             date_time  site_name  posa_continent  user_location_country  \
0  2014-08-11 07:46:59          2               3                     66   
1  2014-08-11 08:22:12          2               3                     66   
2  2014-08-11 08:24:33          2               3                     66   
3  2014-08-09 18:05:16          2               3                     66   
4  2014-08-09 18:08:18          2               3                     66   

   user_location_region  user_location_city  orig_destination_distance  \
0                   348               48862                  2234.2641   
1                   348               48862                  2234.2641   
2                   348               48862                  2234.2641   
3                   442               35390                   913.1932   
4                   442               35390                   913.6259   

   user_id  is_mobile  is_package      ...        srch_children_cnt  \
0       12          0      

In [32]:
# 查看各个属性的具体数据
print(train.info())
# 查看不同属性的具体统计
#print(train.describe())
train.groupby('srch_destination_id').size()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
date_time                    object
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      object
srch_co                      object
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
is_booking                   int64
cnt                          int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
hotel_cluster                int64
dtypes: float64(1), int64(20), object(3)

srch_destination_id
0            2
1           66
2           14
3           15
4         1325
5            1
6            2
7            9
8         2911
9          182
10          45
11         549
13           8
14         530
16         704
17           9
18          39
19        2887
20           1
21       11980
22           5
23          51
24        3478
25        5634
26         231
27        4138
28           4
29           1
30          64
31          12
         ...  
65027       14
65028       16
65031       10
65032        2
65034        1
65035      399
65037        3
65039       20
65041        4
65042       11
65043        1
65047       33
65049        5
65050        3
65051       16
65054        2
65055        2
65056        4
65058        1
65059        1
65060        1
65063        2
65066        1
65068        7
65087        1
65098        3
65102        4
65103        6
65104        3
65107        1
dtype: int64

* date_time： 预订酒店的时间
* site_name： 用户访问的网站名字
* posa_continent：跟网站名字想关联的大洲
* user_location_country,user_location_region,user_location_city：用户所在国家、地区和城市
* orig_destination_distance：用户搜索的时候与酒店的物理距离
* user_id：用户id
* is_mobile：是否使用了电话服务，1表示有
* is_package：是否通过飞行到达酒店
* channel：购买渠道
* srch_ci,srch_co：入住时间和离开时间
* srch_adults_cnt，srch_children_cnt，srch_rm_cnt：成人、小孩和房间数量
* srch_destination_id， srch_destination_type_id：酒店所在地的id以及类型
* is_booking：1表示预订，0表示点击
* cnt：在Expedia网站的session(网站服务器记录用户的会话密匙)相似事件的数量，应该是指用户相似的历史行为
* hotel_continent，hotel_country，hotel_market：酒店所在大洲，国家和market
* hotel_cluster：酒店所在集群id，也是需要预测的属性

查看测试数据

In [31]:
print(test.shape)
print(test.info())
#print(test.head())

(2528243, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2528243 entries, 0 to 2528242
Data columns (total 22 columns):
id                           int64
date_time                    object
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      object
srch_co                      object
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
dtypes: float64(1), int64(18), object(3)
memory usage: 424.4+ MB
None


测试数据少了**`hotel_cluster`**和**'is_booking'**两个属性，前者是需要预测的值，而如果is_booking的值是0，表明是一次点击，值是1表明是一个预定。**测试数据集值包含预定事件，即默认'is_booking = 1'**
另外，测试数据的**'data_time'**是从2015年开始，而训练数据的是在2013-2014两年内的。

查看destination.csv内容

In [35]:
print(destinations.shape)
print(destinations.info())
#print(destinations.head())
#print(destinations.groupby(['srch_destination_id']).size())

(62106, 150)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62106 entries, 0 to 62105
Columns: 150 entries, srch_destination_id to d149
dtypes: float64(149), int64(1)
memory usage: 71.1 MB
None


因此训练数据有3700万条数据，测试数据也有200万行数据。destinations则包含了**srch_destination_id**以及相关的隐含特征，训练数据包含65107个，而该数据中仅有62106，说明有部分酒店是新开的。

In [39]:
# 创建包含所有测试数据集用户id唯一值的集合
test_ids = set(test.user_id.unique())
print('test_user numbers:',len(test_ids))
# 创建包含所有训练数据集用户id唯一值的集合
train_ids = set(train.user_id.unique())
print('train_user numbers:',len(train_ids))
# 计算出有多少测试数据中的用户id在训练集中的用户id中
intersection_count = len(test_ids & train_ids)
# 看看匹配的数目和测试数据集的用户id总数是否一样
print(intersection_count == len(test_ids))

test_user numbers: 1181577
train_user numbers: 1198786
True


训练数据中总共有119万多用户，而测试集也有118万用户，并且都在训练集中出现过

In [43]:
# 查看训练数据中 hotel_cluster，即每个集群中酒店的个数，hotel_cluster就是待预测值
#print(train['hotel_cluster'].value_counts())
#print(train.groupby('hotel_cluster').size())

所以总共有100个酒店集群，也就有100个类，目标是给每个用户预测前5个最有可能的酒店集群

In [44]:
# 将data_time列从object转换成datatime值，这能使它比日期工作起来简单许多。
# 将year和month从data_time提取出来并赋值到它们的列。
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

In [46]:
import random

unique_users = train.user_id.unique()
# 随机抽取10000个用户的数据
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 10000)) ]
sel_train = train[train.user_id.isin(sel_user_ids)]
# 创建新的训练集t1和交叉验证集t2,分别包含2014年8月前的数据和8月后的数据
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]

In [54]:
#如果is_booking的值是0，表明是一次点击，值是1表明是一个预定。
#测试数据集值包含预定事件，所以我们也需要将t2简化成只包含预定
t2 = t2[t2.is_booking == True]
print(t2.shape)

(8196, 26)


一个简单的测试例子，使用mapk程序来计算误差

In [55]:
# 展示训练数据集中一个包含五个最常见的集群的列表
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)
print(most_common_clusters)
# 创建一个元素和t2的行数一样多的列表，每个元素等同于most_common_clusters
predictions = [most_common_clusters for i in range(t2.shape[0])]
print(len(predictions))
# 计算Mean Average Precision
import average_precision as ap

target = [[l] for l in t2["hotel_cluster"]]
print(ap.mapk(target, predictions, k=5))

[91, 41, 48, 64, 65]
8196
0.0629738083618


In [57]:
# 找是否有和hotel_cluster非常相关的元素
#print(train.corr()["hotel_cluster"])

In [58]:
from sklearn.decomposition import PCA
# 保留数据集中的3列数据
pca = PCA(n_components=3)
# 将d1-d149列转换成3列
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]

In [59]:
def calc_fast_features(df):
    """
    生成新的像data_time的数据特征列。
    删除像data_time的非数据列。
    从dest_small添加特征。
    """
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")

    props = {}
    # 生成新的像data_time的数据特征列, 如"month", "day", "hour", "minute", "dayofweek", "quarter"
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)

    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    #print(carryover)
    # 结合其他非时间特征
    for prop in carryover:
        props[prop] = df[prop]
        
    date_props = ["month", "day", "dayofweek", "quarter"]
    # 生成跟住入和离开时间的时间特征生成跟住入和离开时间的时间特征
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')

    ret = pd.DataFrame(props)
    # 从dest_small添加特征
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")
    ret = ret.drop("srch_destination_iddest", axis=1)
    return ret

In [60]:
df = calc_fast_features(t1)
# 用-1替换所有的缺失值
df.fillna(-1, inplace=True)

/home/cai/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cai/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cai/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

In [61]:
# 用随机森林算法来生成预测, 使用3折交叉验证使用训练数据集来生成一个可靠的误差估计
predictors = [c for c in df.columns if c not in ["hotel_cluster"]]
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
scores = cross_validation.cross_val_score(clf, df[predictors], df['hotel_cluster'], cv=3)
print(scores)

/home/cai/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[ 0.06807258  0.06869449  0.06643914]


In [67]:
# 聚合'orig_destination_distance'会为每一个目标找到最受欢迎的酒店集群，
#然后我们就能够预测某个用户搜索的目标是最受欢迎的酒店集群中的一个
def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']

# 通过srch_destination_id，hotel_cluster将t1分组
groups = t1.groupby(cluster_cols)
#print(groups.size())

In [71]:
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    score = bookings + .15 * clicks

    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

4 {14: 0.15, 22: 1.3, 29: 0.15, 30: 0.3, 32: 0.44999999999999996, 38: 2.6, 40: 0.44999999999999996, 53: 0.15, 62: 0.15, 78: 1.75, 81: 0.15, 85: 0.3, 89: 0.15, 93: 0.3, 99: 1.3}


这样就得到每个**srch_destination_id**对应的酒店集群，以及该酒店集群在该srch_destination_id中的权值，这个权值则根据对应的事件是预订还是点击来得到。下面则分别找到排在前五的酒店集群

In [72]:
import operator
# 变换 top_clusters 这个字典来找到每个'srch_destination_id'的前五个酒店集群
cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    # 找到键中的最高的5个集群
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top
for k, v in cluster_dict.items():
    print(k,v)
    break

4 [38, 78, 22, 99, 32]


开始进行预测

In [92]:
preds = []
for index, row in t2.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])
#print('preds=', preds)
print(preds[:5])
# 使用mapk函数来计算准确率
print(ap.mapk([[l] for l in t2["hotel_cluster"]], preds, k = 5))

[[83, 40, 59, 14, 21], [47, 23, 51, 5, 42], [], [0, 84, 41, 96, 33], [97]]
0.230264763299


预测也是迭代t2数据集的每一行，找到每行的srch_destination_id值，通过这个数值来查找之前生成的字典**cluster_dict**,如果存在这个id键值，就将其键值对应的内容，也就是5个酒店集群作为这一行的预测。
通过计算误差的方法可以得到

接下来将使用另一种方法

In [111]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', \
              'hotel_market', 'orig_destination_distance']
# 将训练数据集以匹配的列切分分组
groups = t1.groupby(match_cols)
#print(group)
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    #print('index:',index)
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    #print('group:',group)
    #print('clus:',clus)
    return clus

In [113]:
exact_matches = []
for i in range(t2.shape[0]):
    #print(i)
    exact_matches.append(generate_exact_matches(t2.iloc[i], match_cols))
#print(exact_matches)

**exact_matches**是根据给定的用户所在地三个特征加酒店的2个特征对训练集分组，然后在测试集中找到这5个特征的数值，并根据它们去训练集分好的组中寻找是否有符合的，如果有就返回该小组的**hotel_cluster**数值。这相当于在测试集中找到匹配数据集的用户。

In [120]:
# 展示训练数据集中一个包含五个最常见的集群的列表
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)
# 展示训练集中包含最多酒店数量的5个 hotel_cluster
#print(most_common_clusters)

[91, 41, 48, 64, 65]


In [119]:
# 使用f5函数在给定序列只选取唯一的数值，并且按原来顺序排列，如给定[1,2,2,3,2,5,4]将返回[1,2,3,5,4]
def f5(seq, idfun=None):
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]
print(ap.mapk([[l] for l in t2['hotel_cluster']], preds, k = 5))

0.230264763299


所以最终是通过特征工程选取特定的特征来进行预测得到的结果。
* 第一个是根据**destination_id**和**hotel_cluster**两个特征值，根据**destination_id**在训练集曾经所在的**hotel_cluster**,以及用户历史的操作，即点击还是预订事件来计算权重，生成一个字典，并且选择前5个**hotel_cluster**，测试集只需要找到对应的**destination_id**值，然后查找字典即可得到**hotel_cluster**预测值。
* 第二个就是利用**'user_location_country', 'user_location_region', 'user_location_city','hotel_market', 'orig_destination_distance'**5个特征来对训练集分组，然后在测试集也是根据这5个特征来匹配用户，找到训练集对应的用户，并使用其**hotel_cluster**作为预测值。
* **most_common_clusters**则是简单找到在训练集中包含最多酒店数量的5个 hotel_cluster。

最后就算保存预测数据，格式是csv

In [ ]:
# save to csv file
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = ["{0},{1}".format(t2["id"][i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

end = time.time()
print('totally time is ', end-start_time)